In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import os
# from small_script.myFunctions import *

plt.rcParams['figure.figsize'] = [16.18033, 10]    #golden ratio
plt.rcParams['figure.facecolor'] = 'w'
plt.rcParams['figure.dpi'] = 100

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
data = pd.read_csv("/Users/weilu/Research/database/membrane_training_set/proteins-2019-05-01.csv")

In [22]:
data.pdbid = data.pdbid.apply(lambda x: x[2:-1])

In [23]:
data.head()

,id,ordering,family_name_cache,species_name_cache,membrane_name_cache,name,description,comments,pdbid,resolution,...,species_id,family_id,superfamily_id,classtype_id,type_id,secondary_representations_count,structure_subunits_count,citations_count,created_at,updated_at
0,1,2024.0,OmpA family,Escherichia coli,Gram-neg. outer,"Outer membrane protein A (OMPA), disordered loops",NaN,OmpA is required for the action of colicins K ...,1qjp,1.65,...,9,34,26,2,1,3,1,2,2018-08-13 03:49:46 UTC,2018-09-21 18:14:03 UTC
1,2,2028.0,Enterobacterial Ail/Lom protein,Escherichia coli,Gram-neg. outer,Outer membrane protein X (OMPX),NaN,OmpX from Escherichia coli promotes adhesion t...,1qj8,1.90,...,9,355,26,2,1,7,1,1,2018-08-13 03:49:46 UTC,2018-09-21 18:14:03 UTC
2,3,2033.0,Opacity porins,Neisseria meningitidis,Gram-neg. outer,Outer membrane protein NspA,NaN,Pathogenic Neisseria spp. possess a repertoire...,1p4t,2.55,...,24,337,235,2,1,0,1,0,2018-08-13 03:49:46 UTC,2018-09-21 18:14:03 UTC
3,4,1740.0,Influenza virus matrix protein 2,Influenza virus,Viral,"M2 proton channel of Influenza A, closed state...",NaN,NaN,3lbw,1.65,...,51,263,185,11,1,3,4,0,2018-08-13 03:49:46 UTC,2018-10-02 17:42:36 UTC
4,5,2045.0,"OM protease omptin, OMPT",Yersinia pestis,Gram-neg. outer,Plasminogen activator PLA (coagulase/fibrinoly...,NaN,NaN,2x55,1.85,...,299,36,27,2,1,2,1,0,2018-08-13 03:49:46 UTC,2018-09-21 18:14:03 UTC


In [24]:
data.query("classtype_id == 1")["superfamily_id"].unique().shape

(120,)

# class 1 is Alpha-helical polytopic

In [25]:
d = data.query("classtype_id == 1")

In [27]:
dd = d.groupby("superfamily_id").apply(pd.DataFrame.sample, 1).reset_index(drop=True)

In [32]:
dd.to_csv("/Users/weilu/Research/database/membrane_training_set/chosen.csv")

In [137]:
dd.shape

(120, 31)

In [141]:
dd2.shape

(600, 31)

In [28]:
test = dd.sample(10)

In [31]:
print(test.pdbid.tolist())

['6c70', '5azb', '6mlu', '4r1i', '6bvg', '4zr1', '5o5e', '1lv7', '6eu6', '4pgr']


In [ ]:
pd.DataFrame.sample

In [9]:
t = data["superfamily_id"]

In [ ]:
t.unique()

# extract transmembrane region.

In [60]:
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.Polypeptide import PPBuilder


In [174]:
def get_inside_or_not_table(pdb_file):
    parser = PDBParser(PERMISSIVE=1,QUIET=True)
    structure = parser.get_structure('X', pdb_file)
    inside_or_not_table = []
    for res in structure.get_residues():
        if res.get_id()[0] != " ":
            continue# skip
        try:
            res["CA"].get_vector()
        except:
            print(res.get_id())
        inside_or_not_table.append(int(abs(res["CA"].get_vector()[-1]) < 15))
    return inside_or_not_table

In [175]:
pdb_list = ['6c70', '5azb', '6mlu', '4r1i', '6bvg', '4zr1', '5o5e', '1lv7', '6eu6', '4pgr']

In [178]:
for pdb in pdb_list:
    location = pre + f"../Alpha-helical_polytopic/{pdb}.pdb"
    a = get_inside_or_not_table(location)
    ratio = sum(a)/len(a)
    if ratio < 0.4:
        print("not good", pdb, ratio)
    print(pdb, ratio)

6c70 0.4536082474226804
5azb 0.6232394366197183
not good 6mlu 0.07516339869281045
6mlu 0.07516339869281045
4r1i 0.5735439289239882
6bvg 0.6767337807606264
4zr1 0.48905109489051096
5o5e 0.5853018372703412
not good 1lv7 0.00398406374501992
1lv7 0.00398406374501992
6eu6 0.6469135802469136
4pgr 0.7536231884057971


In [142]:
def extractTransmembrane(toLocation, location):
    x = PDBParser().get_structure("x", location)
    class Transmembrane(Select):
        def accept_residue(self, residue):
            if abs(residue["CA"].get_vector()[-1]) < 15:
                return 1
            else:
                return 0

    io = PDBIO()
    io.set_structure(x)
    io.save(toLocation, Transmembrane())

In [145]:
for pdb in pdb_list:
    pre = "/Users/weilu/Research/database/membrane_training_set/"
    toLocation = pre + f"transmembrane/{pdb}.pdb"
    location = pre + f"cleaned_pdbs/{pdb}.pdb"
    extractTransmembrane(toLocation, location)

In [180]:
pdb_list = ['6c70', '5azb', '4r1i', '6bvg', '4zr1', '5o5e', '6eu6', '4pgr']
for pdb in pdb_list:
    toLocation = pre + f"transmembrane/{pdb}.pdb"
    seq = getSeqFromPDB(toLocation)
    gxxxg_count = 0
    for i in range(len(seq)-4):
        if seq[i] == "G" and seq[i+4] == "A" and " " not in seq[i:i+4]:
            gxxxg_count += 1
            print(pdb, "Found GXXXG", i, seq[i:i+5])

6c70 Found GXXXG 87 GHVFA
5azb Found GXXXG 22 GFIFA
4r1i Found GXXXG 58 GVGIA
4r1i Found GXXXG 86 GGYSA
4r1i Found GXXXG 136 GLIWA
4r1i Found GXXXG 256 GSASA
6bvg Found GXXXG 62 GGSWA
6bvg Found GXXXG 71 GGAFA
6bvg Found GXXXG 72 GAFAA
6bvg Found GXXXG 109 GFLGA
6bvg Found GXXXG 164 GRWIA
6bvg Found GXXXG 236 GSTAA
6bvg Found GXXXG 266 GLAFA
5o5e Found GXXXG 48 GALLA
5o5e Found GXXXG 175 GMTFA
6eu6 Found GXXXG 117 GGWFA
6eu6 Found GXXXG 256 GVIVA
4pgr Found GXXXG 84 GTIGA


In [181]:
for pdb in pdb_list:
    toLocation = pre + f"transmembrane/{pdb}.pdb"
    seq = getSeqFromPDB(toLocation)
    gxxxg_count = 0
    for i in range(len(seq)-4):
        if seq[i] == "G" and seq[i+4] == "G" and " " not in seq[i:i+4]:
            gxxxg_count += 1
            print(pdb, "Found GXXXG", i, seq[i:i+5])

5azb Found GXXXG 38 GVFLG
5azb Found GXXXG 42 GGRIG
5azb Found GXXXG 75 GGLIG
5azb Found GXXXG 97 GLGAG
5azb Found GXXXG 109 GELWG
4r1i Found GXXXG 83 GVSGG
4r1i Found GXXXG 174 GIVYG
6bvg Found GXXXG 28 GKLIG
6bvg Found GXXXG 101 GVFVG
6bvg Found GXXXG 105 GIITG
6bvg Found GXXXG 232 GQVIG
4zr1 Found GXXXG 116 GFAGG
4zr1 Found GXXXG 119 GGLFG
5o5e Found GXXXG 23 GVISG
5o5e Found GXXXG 122 GLEAG
6eu6 Found GXXXG 143 GFNGG
6eu6 Found GXXXG 205 GLITG
6eu6 Found GXXXG 232 GGVIG
4pgr Found GXXXG 14 GLFIG


In [183]:
def count_general_amino_acid_combinations(number_of_amino_acid_types, number_of_sites):
    number_of_valid_codes = 0
    possible_codes = list(set(itertools.combinations_with_replacement(list(range(number_of_sites+1))*number_of_amino_acid_types, number_of_amino_acid_types)))
    for code in possible_codes:
        if sum(list(code)) == number_of_sites:
            print(code)
            number_of_valid_codes += 1
    print(number_of_valid_codes)

In [64]:
x = PDBParser().get_structure("x", "/Users/weilu/Research/database/membrane_training_set/cleaned_pdbs/6c70.pdb")

In [88]:
seq = getSeqFromPDB("/Users/weilu/Research/database/membrane_training_set/transmembrane/gly_only.pdb")

In [136]:
seq = getSeqFromPDB("/Users/weilu/Research/database/")
gxxxg_count = 0
for i in range(len(seq)):
    if seq[i] == "G" and seq[i+4] == "G" and " " not in seq[i:i+4]:
        gxxxg_count += 1
        print("Found GXXXG", i, seq[i:i+5])

Found GXXXG 257 GDAYG


In [129]:
location = "/Users/weilu/Research/database/membrane_training_set/cleaned_pdbs/6c70.pdb"
location = "/Users/weilu/Research/database/membrane_training_set/transmembrane/gly_only.pdb"
x = PDBParser().get_structure("x", location)
ppb=CaPPBuilder()
seq = ""

In [130]:
list(ppb.build_peptides(x))

[<Polypeptide start=16 end=17>,
 <Polypeptide start=19 end=23>,
 <Polypeptide start=35 end=59>,
 <Polypeptide start=68 end=90>,
 <Polypeptide start=132 end=150>,
 <Polypeptide start=158 end=164>,
 <Polypeptide start=173 end=196>,
 <Polypeptide start=263 end=284>,
 <Polypeptide start=288 end=311>,
 <Polypeptide start=362 end=363>,
 <Polypeptide start=367 end=387>]

In [131]:
seq = ""
preResId = 0
for res in x.get_residues():
    resId = res.get_id()[1]
    if resId != preResId + 1:
        seq += " "
    seq += three_to_one(res.get_resname())
    preResId = resId

In [134]:
def getSeqFromPDB(location, considerGap=True):
    x = PDBParser().get_structure("x", location)
    seq = ""
    preResId = 0
    for res in x.get_residues():
        resId = res.get_id()[1]
        if considerGap and resId != preResId + 1:
            seq += " "
        seq += three_to_one(res.get_resname())
        preResId = resId
    return seq.strip()

In [135]:
getSeqFromPDB(location)

'VM GVGHF PHRIYCIVTLLLLLLQYGMMAVNLM LTANTITMLFFLHPIVKMIYFPV CVAGATIFSVISWTGITFI LMIRTFY GHVFALIYQFYYLVISMAVSNSLD VALLLHMLTTTITLTLLAYQAT V GVNVYAATVIGYLLYTLGQVFLFC G FF LDLFASVLGAVVTYFMVLVQL'

In [133]:
seq.strip()

'VM GVGHF PHRIYCIVTLLLLLLQYGMMAVNLM LTANTITMLFFLHPIVKMIYFPV CVAGATIFSVISWTGITFI LMIRTFY GHVFALIYQFYYLVISMAVSNSLD VALLLHMLTTTITLTLLAYQAT V GVNVYAATVIGYLLYTLGQVFLFC G FF LDLFASVLGAVVTYFMVLVQL'

In [125]:
res.get_id()[1]

388

In [121]:
three_to_one(res.get_resname())

'K'

In [118]:
from Bio.PDB.Polypeptide import three_to_one

In [120]:
three_to_one("LYS")

'K'

In [116]:
res.get_id()[1]

388

In [112]:
res.full_id[2]

'A'

In [113]:
seq

'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'

In [98]:
aseq

'FKHQGLVADLLPNIRVMQGVGHFMFNYYSEGKKFPHRIYCIVTLLLLLLQYGMMAVNLMMESDDVDDLTANTITMLFFLHPIVKMIYFPVRSKIFYKTLAIWNNPNSHPLFAESNARFHALAITKMRRLLFCVAGATIFSVISWTGITFIEDS PIPRLMIRTFYPFNAMSGAGHVFALIYQFYYLVISMAVSNSLDVLFCSWLLFACEQLQHLKAIMKPLMELSAT GLTKKQEMLVRSAIKYWVERHKHVVRLVTAVGDAYGVALLLHMLTTTITLTLLAYQATKVNGVNVYAATVIGYLLYTLGQVFLFCIFGNRLIEESSSVMEAAYSCHWYDGSEEAKTFVQIVCQQCQKAMSISGAKFFTVSLDLFASVLGAVVTYFMVLVQLK'

In [99]:
for i in range(len(seq)):
    if seq[i] == " ":
        print(i)

153
227


In [90]:
gxxxg_count

0

In [96]:
def getSeqFromPDB(location, considerGap=True):
    x = PDBParser().get_structure("x", location)
    ppb=PPBuilder()
    seq = ""
    for pp in ppb.build_peptides(x):
        if considerGap:
            seq += " " + str(pp.get_sequence())
        else:
            seq += str(pp.get_sequence())
    return seq.strip()

In [82]:
len(seq)

388

In [72]:
a = app.get_sequence()

In [70]:
str(pp.get_sequence())

'GLTKKQEMLVRSAIKYWVERHKHVVRLVTAVGDAYGVALLLHMLTTTITLTLLAYQATKVNGVNVYAATVIGYLLYTLGQVFLFCIFGNRLIEESSSVMEAAYSCHWYDGSEEAKTFVQIVCQQCQKAMSISGAKFFTVSLDLFASVLGAVVTYFMVLVQLK'

In [41]:
res = list(x.get_residues())[0]

In [42]:
ca = res["CA"]

In [45]:
if abs(ca.get_vector()[-1]) > 15

SyntaxError: invalid syntax (<ipython-input-45-4d008d927f5f>, line 1)

In [46]:
from Bio.PDB.PDBIO import PDBIO

In [53]:
pre = "/Users/weilu/Research/database/membrane_training_set/transmembrane/"

In [54]:
from Bio.PDB.PDBIO import Select

In [55]:
class Transmembrane(Select):
    def accept_residue(self, residue):
        if abs(residue["CA"].get_vector()[-1]) < 15:
            return 1
        else:
            return 0

io = PDBIO()
io.set_structure(x)
io.save(pre + 'gly_only.pdb', Transmembrane())

In [56]:
res.get_id()

(' ', 1, ' ')

In [57]:
x = PDBParser().get_structure("x", pre + 'gly_only.pdb')

In [58]:
res = list(x.get_residues())[50]

In [59]:
res.get_id()

(' ', 86, ' ')